# Runs for Micro-wrapper development

In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY311,
)

In [2]:
import os
import subprocess
import sys

sys.path.insert(1, os.path.abspath(".."))

import GooseSLURM as gs
import numpy as np

import lysis

In [3]:
scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("total_molecules", int),
        ("pore_size", float),  # in microns. Code requires cm
        ("fiber_diameter", float),  # in microns. Needs to be added to pore_size to get grid_node_distance
        ("binding_sites", int),
        ("cols", int),
        ("rows", int),
        ("empty_rows", int),
        ("forced_unbind", float),
    ]
)
mechanism_type = np.dtype([("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("scenario", np.str_, 40),
        ("mechanism", np.str_, 40),
        ("seed", np.int64),
    ]
)
diameter_code = {
    46: "Q0",
    72.7: "Q2",
    105.1: "TF",
    123.0: "TB",
    145.4: "Q4",
}

In [4]:
scenarios = np.array(
    [
        (),
    ],
    dtype=scenario_type,
)
mechanisms = np.array(
    [
        ("Into and along - External", "macro_diffuse_into_and_along__external"),
    ],
    dtype=mechanism_type,
)

In [5]:
in_file_code = "_PLG2_tPA01_{input_code}.dat"
out_file_code = "_{scenario_code}"

In [6]:
# seed sequence entropy: 234567848524899077628396107049383963466
group_code = "2024-04-16-19"
runs = np.array(
    [
        # (group_code + "00", "TB-xi__1_582_867", "Into and along - External", 4_133_290_200, 0, 10, 1),
        # (group_code + "01", "TB-xi__949_720", "Into and along - External", 423_058_538, 0, 10, 1),
        # (group_code + "02", "TB-xi__316_573", "Into and along - External", 684_840_937, 0, 10, 1),
        # (group_code + "03", "TF-x__746_330", "Into and along - External", 868_609_522, 0, 10, 1),
        # (group_code + "04", "TF-x__447_798", "Into and along - External", 3_835_132_626, 0, 10, 1),
        # (group_code + "05", "TF-x__149_266", "Into and along - External", 153_757_143, 0, 10, 1),
        # (group_code + "06", "TF-vii__746_330", "Into and along - External", 456_534_494, 0, 10, 1),
        # (group_code + "07", "TF-vii__447_798", "Into and along - External", 3_326_749_085, 0, 10, 1),
        # (group_code + "08", "TF-vii__149_266", "Into and along - External", 3_225_783_680, 0, 10, 1),
        # (group_code + "09", "TF-v__746_330", "Into and along - External", 1_705_037_629, 0, 10, 1),
        # (group_code + "10", "TF-v__447_798", "Into and along - External", 22_857_604, 0, 10, 1),
        # (group_code + "11", "TF-v__149_266", "Into and along - External", 3_425_717_575, 0, 10, 1),
        (group_code + "12", "TB-xi__21_105", "Into and along - External", 2_061_649_767, 0, 10, 1),
        (group_code + "13", "TF-x__9_951", "Into and along - External", 3_687_972_975, 0, 10, 1),
        (group_code + "14", "TF-vii__9_951", "Into and along - External", 1_422_833_478, 0, 10, 1),
        (group_code + "15", "TF-v__9_951", "Into and along - External", 77_748_149, 0, 10, 1),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    p = {
        "total_molecules": int(scen["total_molecules"]),
        "pore_size": scen["pore_size"] / 10_000,  # convert to cm
        "grid_node_distance": scen["fiber_diameter"] / 1_000 + scen["pore_size"],
        "binding_sites": int(scen["binding_sites"]),
        "cols": int(scen["cols"]),
        "rows": int(scen["rows"]),
        "empty_rows": int(scen["empty_rows"]),
        "forced_unbind": scen["forced_unbind"],
        "seed": int(run["seed"]),
        "total_time": int(run["running_time"]),
        "save_interval": int(run["save_interval"]),
        "macro_version": mech["executable"],
    }
    e.initialize_macro_param(p)
    e.to_file()

    with open(os.path.join(e.os_path, "README.md"), "w") as file:
        file.write(
            f""" ## {e.experiment_code}
Lysis Front study
Array version
New scenarios from bench experiments

Uses Fortran Macroscale code with
    -  Diffuse into and along
    -  'restricted move' bug correction
    -  'passerby molecule' bug correction
    -  addition of extra output data
    -  changing the termination criteria from 'fixed time' to 'all fibers degraded'
    -  changing macro-unbind wait time to remaining leaving time.
    -  eliminating unused data variables
    -  Moved degraded fiber check into molecule loop
    -  Removed "degrade" array and use "t_degrade" instead
    -  Read in "neighborc" array generated in Python
    -  Output molecule bind/unbind times for transit time calculations
    -  Output molecule bind/unbind locations.

This set of data is from the "{run['scenario']}" set.
"""
        )

    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", mech["executable"] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1912'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1913'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1914'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1915'], returncode=0, stdout=b'', stderr=b'')


In [8]:
folder_vars = """
homedir="/home/bpaynter/git/UCO-OpResearch/lysis"
workdir="/tmp/bpaynter/${SLURM_ARRAY_JOB_ID}-${SLURM_ARRAY_TASK_ID}"
datadir="${workdir}/data/${exp_code}"
"""

transfer_script = """
# 1. Transfer to node 
# ====================
echo "Starting data transfer to node."

# create/empty the temporary directory on the compute node
if [ ! -d "${workdir}" ]; then
  mkdir -p "${workdir}"
else
  echo rm -ri "${workdir}"/*
fi

mkdir -p "${datadir}"

cp ${homedir}/data/${exp_code}/*${in_code} ${datadir}/
cp ${homedir}/data/${exp_code}/params.json ${datadir}/
cp ${homedir}/bin/${fort_executable} ${workdir}/
echo "Data transfer to node completed."
"""

return_script = """
# 2. Function to transfer back to the head node 
# ==============================================

# define clean-up function
function clean_up {
  sim=$(printf "%02d" ${SLURM_ARRAY_TASK_ID})
  # - copy everything from the temporary directory on the compute-node
  echo "Copying data back to home directory."
  mkdir -p ${homedir}/data/${exp_code}/$sim
  cp -prf "${datadir}"/*${out_code}* ${homedir}/data/${exp_code}/$sim/
  cp -prf "${datadir}"/*.txt ${homedir}/data/${exp_code}/$sim/
  # - erase the temporary directory from the compute-node
  echo "Erasing temporary directories."
  rm -rf "${workdir}"/*
  rm -rf "${workdir}"
  # - exit the script
  echo "Job complete. Exiting."
  exit
}

# call "clean_up" function when this script exits, it is run even if SLURM cancels the job
trap 'clean_up' EXIT
"""
execute_script = """
# 3. Execute 
# ===========

source /home/bpaynter/.bashrc
source /home/bpaynter/lysis.sh
cd ${homedir}/src/python
echo "Starting code execution."

python -u fortran_run.py \
    --in_code ${in_code} \
    --out_code ${out_code}.dat \
    -n $SLURM_ARRAY_TASK_ID \
    --cwd ${workdir} \
    ${workdir}/${fort_executable} \
    ${exp_code}
    
sleep 10
echo "Code execution complete."
"""

In [9]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    e.read_file()
    run_vars = f"""
exp_code="{e.experiment_code}"
in_code="{in_file_code.format(input_code=run["scenario"].split("_")[0])}"
out_code="{out_file_code.format(scenario_code=run['scenario'])}"
fort_executable="{mech['executable']}"
echo "Experiment ${{exp_code}} (${{out_code}}), Simulation #${{SLURM_ARRAY_TASK_ID}}"
"""
    # job-options
    sbatch = {
        "job-name": f"lysis-{e.experiment_code}",
        "out": os.path.join(e.os_path, "job.slurm-%A-%a.out"),
        "array": f"0-{e.macro_params.total_trials-1}",
        "nodes": 1,
        "mem": 3096,
        "ntasks": 1,
        "cpus-per-task": 1,
        "exclusive=user": "",
        # "exclusive": "",
        "partition": "long",
    }
    script = [run_vars, folder_vars, transfer_script, return_script, execute_script]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1912/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223365\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1913/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223366\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1914/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223367\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1915/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223368\n', stderr=b'')
